# To Does:

- Find out how to break down multiple recipients in one cell into multiple rows while keeping the sender 
- Find out how to extract email body
- Find out how to create a dataframe or dictionary with sender/ recipient information (how many emails has each employee send to and received from all other employees)

## Data source:

Kaggle, The Enron Email Dataset: https://www.kaggle.com/wcukierski/enron-email-dataset
based on the May 7, 2015 Version of dataset, as published at https://www.cs.cmu.edu/~./enron/

In [1]:
# libraries for pre-processing
import pandas as pd
import numpy as np
import re
# libraries for network analysis
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('../data/emails.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [4]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
df.message[0]

"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n "

In [6]:
print(df.message[0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [7]:
df.file[0]

'allen-p/_sent_mail/1.'

In [8]:
print(df.message[268900])

Message-ID: <12445037.1075840220349.JavaMail.evans@thyme>
Date: Thu, 2 Nov 2000 03:40:00 -0800 (PST)
From: cscusack@email.msn.com
To: kenneth.lay@enron.com
Subject: Ken Derr
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: "cscusack" <cscusack@email.msn.com>
X-To: <kenneth.lay@enron.com>
X-cc: 
X-bcc: 
X-Folder: \Kenneth_Lay_Dec2000\Notes Folders\All documents
X-Origin: LAY-K
X-FileName: klay.nsf

Rosalee, just heard back from Mr. Derr's assistant.  Unfortunately, he is 
unable to attend. Please delete his name from the  letter/invitation. Thank 
you. Chris


### Take a random sample for testing purposes in the pre-processing stage

In [50]:
test = df.sample(n=200, random_state=0)

## Define helper functions for pre-processing the data

Note:
Figure out a way to drop rows based on certain folder names in the file column.

To drop:
- calendar
- schedule_crawler
- funny

Maybe to drop:
- press_releases
- contact
- contacts
- private* (including variations such as private_folders)
- *personal* (including variations such as my_personal)
- tasks
- to_do

In [11]:
def get_user(df):
    """
    Input:
        DataFrame
    Function:
        Extracts the user account in the column 'file' of the DataFrame and appends it to a list. Appends this 
        list as column 'user' to the dataframe and returns it.
    Output:
        DataFrame
    """
    file_split = []
    
    for file in df['file']:
        split = file.rsplit('/')
        file_split.append(split[0])
        
    df['user'] = file_split
        
    return df

In [55]:
# helper function to clean up string in column "message"
def clean_text(message, match):
    """
    Takes a string and the separater to be replaced, replaces linebrakes with empty quotation marks and splits 
    the string at those occurences. Returns a string.
    """ 
    text = re.sub(r"\n\t", "", message)
    return re.split(match, message)

In [13]:
def get_date_time(df):
    """
    Input:
        DataFrame
    Function:
        Extracts the date and time from the column 'message_clean' of the DataFrame, stores the information in
        seperate columns and returns the DataFrame.
    Output:
        DataFrame
    """
    df['date'] = df['message_clean'].apply(lambda x: x[1])
    df.loc[:, 'date'] = df.loc[:, 'date'].apply(lambda x: x[6:31])
    df['date'] = pd.to_datetime(df['date'])
    
    return df

In [14]:
# helper function to extract email addresses from string
def extract_email_string(message, substring):
    """
    Takes a large string (i.e. message) and a substring (i.e. indicator for sender or recipient) to find all
    occurences of a defined regular expression and stores them in a list. If substring is not found in message,
    then the list item remains empty. Returns a list.
    """
    result = re.findall('[\w\.-]+@[\w\.-]+\.\w+', message)
    if substring not in message:
        result = ""
    return result

In [ ]:
def get_sender_recipients(df):
    """
    Input:
        DataFrame
    Function:
        Extracts the email addresses of sender, recipient, recipient (cc) and recipient (bcc) from the column 
        'message_clean' of the DataFrame, stores the information in seperate columns and returns the DataFrame.
    Output:
        DataFrame
    """
    
    df['sender'] = df['message_clean'].apply(lambda x : x[2])
    df.loc[:, 'sender'] = df.loc[:,'sender'].apply(lambda x : x[6:])
    
    df['recipients'] = df['message_clean'].apply(lambda x : x[3])
    df.loc[:, 'recipients'] = df.loc[:, 'recipients'].apply(lambda x: x[3:])
    
    test['recipients_cc'] = test['message_clean'].apply(lambda x : x[5])
    
    test['recipients_bcc'] = test['message_clean'].apply(lambda x : x[9])
    
    return df

In [22]:
test['sender'] = test['message_clean'].apply(lambda x : x[2])
test.loc[:, 'sender'] = test.loc[:,'sender'].apply(lambda x : x[6:])

In [23]:
test['recipient'] = test['message_clean'].apply(lambda x : x[3])
test.loc[:, 'recipient'] = test.loc[:, 'recipient'].apply(lambda x: x[3:])

In [24]:
test['recipient_cc'] = test['message_clean'].apply(lambda x : x[5])

In [25]:
test['recipient_bcc'] = test['message_clean'].apply(lambda x : x[9])

In [42]:
test['test_message'] = test['message_clean'].apply(lambda x : x[3:])

In [40]:
mails = test['message_clean'].str.extractall(r'(\w+@\w+\.\w+)')

In [46]:
test['test_message'] = test['test_message'].str.replace(r'\t','')

In [57]:
test

,file,message,message_clean
510816,williams-w3/schedule_crawler/1387.,Message-ID: <20978984.1075839990072.JavaMail.e...,[Message-ID: <20978984.1075839990072.JavaMail....
346872,nemec-g/all_documents/5431.,Message-ID: <8987828.1075842774735.JavaMail.ev...,[Message-ID: <8987828.1075842774735.JavaMail.e...
158351,hain-m/all_documents/842.,Message-ID: <30686869.1075860369073.JavaMail.e...,[Message-ID: <30686869.1075860369073.JavaMail....
351661,nemec-g/notes_inbox/2544.,Message-ID: <4155293.1075842841039.JavaMail.ev...,[Message-ID: <4155293.1075842841039.JavaMail.e...
20325,baughman-d/power/legal_agreements/96.,Message-ID: <12670521.1075862605089.JavaMail.e...,[Message-ID: <12670521.1075862605089.JavaMail....
...,...,...,...
245520,kean-s/discussion_threads/1931.,Message-ID: <22943543.1075848122704.JavaMail.e...,[Message-ID: <22943543.1075848122704.JavaMail....
487224,tholt-j/discussion_threads/208.,Message-ID: <29143432.1075857489662.JavaMail.e...,[Message-ID: <29143432.1075857489662.JavaMail....
411055,semperger-c/deleted_items/143.,Message-ID: <8864897.1075841547320.JavaMail.ev...,[Message-ID: <8864897.1075841547320.JavaMail.e...
17350,bass-e/sent/623.,Message-ID: <22191133.1075854699828.JavaMail.e...,[Message-ID: <22191133.1075854699828.JavaMail....


In [74]:
df = test['message'].str.extractall(r'To:([\w\.-]+@[\w\.-]+\.\w+)')

In [75]:
df

,,0
,match,


In [71]:
test['message'][510816]

'Message-ID: <20978984.1075839990072.JavaMail.evans@thyme>\nDate: Thu, 13 Dec 2001 06:37:39 -0800 (PST)\nFrom: pete.davis@enron.com\nTo: pete.davis@enron.com\nSubject: Start Date: 12/13/01; HourAhead hour: 5;\nCc: albert.meyers@enron.com, bill.williams@enron.com, craig.dean@enron.com, \n\tgeir.solberg@enron.com, john.anderson@enron.com, \n\tmark.guzman@enron.com, michael.mier@enron.com, pete.davis@enron.com, \n\tryan.slinger@enron.com\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nBcc: albert.meyers@enron.com, bill.williams@enron.com, craig.dean@enron.com, \n\tgeir.solberg@enron.com, john.anderson@enron.com, \n\tmark.guzman@enron.com, michael.mier@enron.com, pete.davis@enron.com, \n\tryan.slinger@enron.com\nX-From: Davis, Pete </O=ENRON/OU=NA/CN=RECIPIENTS/CN=PDAVIS1>\nX-To: Davis, Pete </O=ENRON/OU=NA/CN=RECIPIENTS/CN=PDAVIS1>\nX-cc: Meyers, Albert </O=ENRON/OU=NA/CN=RECIPIENTS/CN=BMEYERS>, Williams III, Bill </O=ENRON/OU=NA/CN=RECIPIE

In [202]:
test.loc[:, 'recipient_cc'] = test.loc[:, 'recipient_cc'].apply(lambda x: extract_email_string(x, 'Cc:'))

In [208]:
test.loc[:, 'recipient_bcc'] = test.loc[:, 'recipient_bcc'].apply(lambda x: extract_email_string(x, 'Bcc:'))

### Test helper functions for functionality

In [56]:
# test clean_text function, store results in new column "message_tidy"
test['message_clean'] = test.message.apply(lambda x: clean_text(x, '\n\t'))

In [122]:
# test get_email_account function
test = get_user(test)

In [123]:
# test get_date_time function
test = get_date_time(test)
test

,file,message,message_clean,user,date
510816,williams-w3/schedule_crawler/1387.,Message-ID: <20978984.1075839990072.JavaMail.e...,[Message-ID: <20978984.1075839990072.JavaMail....,williams-w3,2001-12-13 06:37:39
346872,nemec-g/all_documents/5431.,Message-ID: <8987828.1075842774735.JavaMail.ev...,[Message-ID: <8987828.1075842774735.JavaMail.e...,nemec-g,2001-03-20 00:12:00
158351,hain-m/all_documents/842.,Message-ID: <30686869.1075860369073.JavaMail.e...,[Message-ID: <30686869.1075860369073.JavaMail....,hain-m,2001-03-15 01:30:00
351661,nemec-g/notes_inbox/2544.,Message-ID: <4155293.1075842841039.JavaMail.ev...,[Message-ID: <4155293.1075842841039.JavaMail.e...,nemec-g,2001-01-03 10:00:00
20325,baughman-d/power/legal_agreements/96.,Message-ID: <12670521.1075862605089.JavaMail.e...,[Message-ID: <12670521.1075862605089.JavaMail....,baughman-d,2001-09-26 07:59:09
...,...,...,...,...,...
245520,kean-s/discussion_threads/1931.,Message-ID: <22943543.1075848122704.JavaMail.e...,[Message-ID: <22943543.1075848122704.JavaMail....,kean-s,2001-01-01 23:15:00
487224,tholt-j/discussion_threads/208.,Message-ID: <29143432.1075857489662.JavaMail.e...,[Message-ID: <29143432.1075857489662.JavaMail....,tholt-j,2001-01-10 04:30:00
411055,semperger-c/deleted_items/143.,Message-ID: <8864897.1075841547320.JavaMail.ev...,[Message-ID: <8864897.1075841547320.JavaMail.e...,semperger-c,2002-01-25 14:00:56
17350,bass-e/sent/623.,Message-ID: <22191133.1075854699828.JavaMail.e...,[Message-ID: <22191133.1075854699828.JavaMail....,bass-e,2000-06-30 06:04:00


In [196]:
test['message_clean'][510816]

['Message-ID: <20978984.1075839990072.JavaMail.evans@thyme>',
 'Date: Thu, 13 Dec 2001 06:37:39 -0800 (PST)',
 'From: pete.davis@enron.com',
 'To: pete.davis@enron.com',
 'Subject: Start Date: 12/13/01; HourAhead hour: 5;',
 'Cc: albert.meyers@enron.com, bill.williams@enron.com, craig.dean@enron.com, geir.solberg@enron.com, john.anderson@enron.com, mark.guzman@enron.com, michael.mier@enron.com, pete.davis@enron.com, ryan.slinger@enron.com',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'Bcc: albert.meyers@enron.com, bill.williams@enron.com, craig.dean@enron.com, geir.solberg@enron.com, john.anderson@enron.com, mark.guzman@enron.com, michael.mier@enron.com, pete.davis@enron.com, ryan.slinger@enron.com',
 'X-From: Davis, Pete </O=ENRON/OU=NA/CN=RECIPIENTS/CN=PDAVIS1>',
 'X-To: Davis, Pete </O=ENRON/OU=NA/CN=RECIPIENTS/CN=PDAVIS1>',
 'X-cc: Meyers, Albert </O=ENRON/OU=NA/CN=RECIPIENTS/CN=BMEYERS>, Williams III, Bill </O=ENRON/OU=

In [205]:
test['message_clean'][351661]

['Message-ID: <4155293.1075842841039.JavaMail.evans@thyme>',
 'Date: Wed, 3 Jan 2001 10:00:00 -0800 (PST)',
 'From: dwight.beach@enron.com',
 'To: gerald.nemec@enron.com',
 'Subject: GROUND LEASE (ELIZABETHTOWN).doc',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Dwight Beach',
 'X-To: Gerald Nemec',
 'X-cc: ',
 'X-bcc: ',
 'X-Folder: \\Gerald_Nemec_Dec2000_June2001_2\\Notes Folders\\Notes inbox',
 'X-Origin: NEMEC-G',
 'X-FileName: gnemec.nsf',
 '',
 'I think that these are current, but you might want to check with Steve Van ',
 'Hooser to verify that there have been no changes.',
 '',
 '',
 '---------------------- Forwarded by Dwight Beach/HOU/ECT on 01/03/2001 05:59 ',
 'PM ---------------------------',
 '',
 '',
 '"Jay Sonnenberg" <jsonnenberg@bracepatt.com> on 12/22/2000 05:15:48 PM',
 'To: <gmasterson@bassberry.com>',
 'cc: "Clark Thompson" <cthompson@bracepatt.com>, <dbeach@enron.com>, ',
 '<Glenn.Wright@enron.

## Explore data

In [20]:
# show number of emails per unique email account, number of unique email accounts is shown in summary line
emails_per_acc = test['user'].value_counts()
emails_per_acc

dasovich-j      15
kean-s          14
shackleton-s    13
kaminski-v      10
symes-k          9
                ..
haedicke-m       1
dickson-s        1
griffith-j       1
heard-m          1
lokay-m          1
Name: user, Length: 79, dtype: int64

In [ ]:
"""
add a node for each sender and recipient
for address in kaminski_addresses.keys():
    kam_network.add_node(address, size=kaminski_addresses[address])
for recipient in kaminski_recipients.keys():
    kam_network.add_node(recipient, size=kaminski_recipients[recipient])
    
# add an edge for every email send between two employees
for recipient in kaminski_sender.columns:
    kam_network.add_edge(sender, recipient, weight = kaminski_sender[sender][recipient])
    
"""